In [1]:
import numpy as np

# Normalized Cross Correlation
Create sample image, pick a template, find the best matching of that template in the image.

In [71]:
# Sample image.
sample_image = np.zeros((10, 10), dtype=np.uint8)
sample_image[3, 3] = 244
sample_image[3, 4] = 213
sample_image[3, 5] = 222
sample_image[3, 6] = 245
sample_image[4, 4] = 210
sample_image[4, 5] = 202
sample_image[4, 6] = 166
sample_image[5, 4] = 110
sample_image[5, 5] = 112
sample_image[5, 6] = 189
print(sample_image)

[[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0 244 213 222 245   0   0   0]
 [  0   0   0   0 210 202 166   0   0   0]
 [  0   0   0   0 110 112 189   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]]


In [72]:
# Template to match.
template = np.array([[212, 221], [209, 204]])
print(template)

[[212 221]
 [209 204]]


In [73]:
import math

def match_cross_correlation(image: np.ndarray, pattern: np.ndarray) -> np.ndarray:
    '''
    Performs normalized cross correlation. 

    Parameters:
    - image (np.ndarray): Given 2D MxN image.
    - pattern (np.ndarray): 2D pattern to match in the image.

    Returns:
    - np.ndarray: Map with shape of input image, with correlation values.
    '''

    # Allocate return map.
    correlation_map = np.zeros(image.shape)

    # Perform cross-correlation at every pixel, filling the correlation map with values.
    # https://www.youtube.com/watch?v=5YAA7vS6kVU&t=1s 18:19.
    # Cross Correlation Function.
    for u in range(image.shape[0]):
        for v in range(image.shape[1]):
            correlation_map[u, v] = find_cross_correlation_value(image, pattern, u, v)

    return correlation_map

def find_cross_correlation_value(image: np.ndarray, pattern: np.ndarray, offset_u: int, offset_v: int) -> float:
    '''
    Calculates cross-corelation value at a given offset.

    Parameters:
    - image (np.ndarray): Given 2D MxN image.
    - pattern (np.ndarray): 2D pattern to match in the image.
    - offset_u (int): Row offset.
    - offset_v (int): Column offset.

    Returns:
    - int: Cross correlation value at given offset.
    '''
    # Get the overlapping patch from original image.
    # Do not go over image.
    max_u = min(image.shape[0] - 1, offset_u + pattern.shape[0] - 1) + 1
    max_v = min(image.shape[1] - 1, offset_v + pattern.shape[1] - 1) + 1
    overlapping_patch = image[offset_u : max_u, offset_v : max_v]

    # Covariance of g1 and g2 (numerator).
    covariance = 0

    # Find parameters independent of specific pixel.
    pattern_mean = mean(pattern)
    overlapping_patch_mean = mean(overlapping_patch)
    
    # Only the matching shape.
    if overlapping_patch.shape == pattern.shape:    
        for i in range(overlapping_patch.shape[0]):
            for j in range(overlapping_patch.shape[1]):
                covariance += (pattern[i, j] - pattern_mean) * (overlapping_patch[i, j] - mean(overlapping_patch))

        covariance /= overlapping_patch.size - 1

        # Calculate the denominator.
    
        # Standard deviation of the template.
        pattern_standard_deviation = standard_deviation(pattern, pattern_mean)
        overlapping_patch_standard_deviation = standard_deviation(overlapping_patch, overlapping_patch_mean)

        # Normalized Cross Correlation.
        # https://www.youtube.com/watch?v=5YAA7vS6kVU&t=1s 23:39

        numerator = covariance
        denominator = pattern_standard_deviation * overlapping_patch_standard_deviation

        if numerator == 0.0 or denominator == 0.0:
            return 0.0
    
        return covariance / denominator
        
    else:
        return 0.0

def mean(image : np.ndarray) -> float:
    '''
    Finds the mean intensity value of given image.

    Parameters:
    - image (np.ndarray): Given image.

    Returns:
    - float: Mean intensity value of a given image.
    '''

    mean = 0.0

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            mean += image[i, j]

    return mean / image.size

def standard_deviation(image: np.ndarray, mean: float) -> float:
    '''
    Calculates standard deviation of an image.

    Parameters:
    - image (np.ndarray): Given image.
    - mean (float): Precalculated mean of that image.

    Returns:
    - float: Standard deviation.
    '''

    standard_dev = 0
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            standard_dev += (image[i, j] - mean) ** 2

    standard_dev /= image.size - 1

    return math.sqrt(standard_dev)
    

In [85]:
cross_correlation_mask = match_cross_correlation(sample_image, template)

best_match = np.max(cross_correlation_mask)
best_match_index = np.unravel_index(np.argmax(cross_correlation_mask), cross_correlation_mask.shape)
print(f'best match {best_match} at {best_match_index}')
np.set_printoptions(precision=3)
print(cross_correlation_mask)
print(sample_image[best_match_index[0]:best_match_index[0] + 2, best_match_index[1]:best_match_index[1] + 2])

best match 0.9915610305527968 at (3, 4)
[[ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.    -0.7   -0.777 -0.816 -0.826 -0.233  0.     0.     0.   ]
 [ 0.     0.     0.887  0.237  0.992  0.96  -0.111  0.     0.     0.   ]
 [ 0.     0.     0.     0.541  0.772 -0.023 -0.177  0.     0.     0.   ]
 [ 0.     0.     0.     0.887  0.815  0.935  0.047  0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]]
[[213 222]
 [210 202]]


## We should work on normalized image and patterns.

In [95]:
image_sample_mean = mean(sample_image)
image_sample_standard_deviation = standard_deviation(sample_image, image_sample_mean)
pattern_mean = mean(template)
pattern_standard_deviation = standard_deviation(template, pattern_mean)

print(f'image mean: {image_sample_mean}')
print(f'image std: {image_sample_standard_deviation}')
print(f'pattern mean: {pattern_mean}')
print(f'pattern std: {pattern_standard_deviation}')

image mean: 19.13
image std: 59.496482367795565
pattern mean: 211.5
pattern std: 7.14142842854285


In [97]:
# Normalize image and pattern.
sample_image_normalized = np.zeros(sample_image.shape, dtype=float)
pattern_normalized = np.zeros(template.shape)

for u in range(sample_image.shape[0]):
    for v in range(sample_image.shape[1]):
        sample_image_normalized[u, v] = (sample_image[u, v] - image_sample_mean) / image_sample_standard_deviation

for u in range(template.shape[0]):
    for v in range(template.shape[1]):
        pattern_normalized[u, v] = (template[u, v] - pattern_mean) / pattern_standard_deviation

print(sample_image_normalized)
print(pattern_normalized)

[[-0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322]
 [-0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322]
 [-0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322]
 [-0.322 -0.322 -0.322  3.78   3.259  3.41   3.796 -0.322 -0.322 -0.322]
 [-0.322 -0.322 -0.322 -0.322  3.208  3.074  2.469 -0.322 -0.322 -0.322]
 [-0.322 -0.322 -0.322 -0.322  1.527  1.561  2.855 -0.322 -0.322 -0.322]
 [-0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322]
 [-0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322]
 [-0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322]
 [-0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322 -0.322]]
[[ 0.07  1.33]
 [-0.35 -1.05]]


## Or via numpy.

In [98]:
mean_value = np.mean(sample_image)
std_dev_value = np.std(sample_image)
mean_value_pattern = np.mean(template)
std_dev_value_pattern = np.std(template)

sample_image_normalized_np = (sample_image - mean_value) / std_dev_value
pattern_normalized_np = (template - mean_value_pattern) / std_dev_value_pattern

print(sample_image_normalized_np)
print(pattern_normalized_np)

[[-0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323]
 [-0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323]
 [-0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323]
 [-0.323 -0.323 -0.323  3.799  3.275  3.427  3.815 -0.323 -0.323 -0.323]
 [-0.323 -0.323 -0.323 -0.323  3.224  3.089  2.481 -0.323 -0.323 -0.323]
 [-0.323 -0.323 -0.323 -0.323  1.535  1.569  2.87  -0.323 -0.323 -0.323]
 [-0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323]
 [-0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323]
 [-0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323]
 [-0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323 -0.323]]
[[ 0.081  1.536]
 [-0.404 -1.213]]


## Try matching again.

In [101]:
cross_correlation_mask_norm = match_cross_correlation(sample_image_normalized, pattern_normalized)

best_match = np.max(cross_correlation_mask_norm)
best_match_index = np.unravel_index(np.argmax(cross_correlation_mask_norm), cross_correlation_mask_norm.shape)
print(f'best match {best_match} at {best_match_index}')
np.set_printoptions(precision=3)
print(cross_correlation_mask_norm)
print(sample_image_normalized_np[best_match_index[0]:best_match_index[0] + 2, best_match_index[1]:best_match_index[1] + 2])

best match 0.9915610305527968 at (3, 4)
[[ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.    -0.7   -0.777 -0.816 -0.826 -0.233  0.     0.     0.   ]
 [ 0.     0.     0.887  0.237  0.992  0.96  -0.111  0.     0.     0.   ]
 [ 0.     0.     0.     0.541  0.772 -0.023 -0.177  0.     0.     0.   ]
 [ 0.     0.     0.     0.887  0.815  0.935  0.047  0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]]
[[3.275 3.427]
 [3.224 3.089]]


## And via numpy also.

In [103]:
cross_correlation_mask_norm_np = match_cross_correlation(sample_image_normalized_np, pattern_normalized_np)

best_match = np.max(cross_correlation_mask_norm_np)
best_match_index = np.unravel_index(np.argmax(cross_correlation_mask_norm_np), cross_correlation_mask_norm_np.shape)
print(f'best match {best_match} at {best_match_index}')
np.set_printoptions(precision=3)
print(cross_correlation_mask_norm_np)
print(sample_image_normalized_np[best_match_index[0]:best_match_index[0] + 2, best_match_index[1]:best_match_index[1] + 2])

best match 0.9915610305527973 at (3, 4)
[[ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.    -0.7   -0.777 -0.816 -0.826 -0.233  0.     0.     0.   ]
 [ 0.     0.     0.887  0.237  0.992  0.96  -0.111  0.     0.     0.   ]
 [ 0.     0.     0.     0.541  0.772 -0.023 -0.177  0.     0.     0.   ]
 [ 0.     0.     0.     0.887  0.815  0.935  0.047  0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]]
[[3.275 3.427]
 [3.224 3.089]]


## Seems like my implementation of mean and std works correctly.
Also it seems that normal distribution doesn't change anything in this case. It still works the same